In [1]:
import os
import pandas as pd
# Mudança de diretório
os.chdir('E:\\Dropbox\\transito')

In [5]:
import os

# Função para importar arquivos CSV de uma pasta e concatená-los em um único DataFrame
def importar_csv_de_pasta(pasta):
    # Listar todos os arquivos na pasta que terminam com '.csv'
    arquivos_csv = [f for f in os.listdir(pasta) if f.endswith('.csv')]
    
    # Criar uma lista para armazenar os DataFrames
    dataframes = []

    # Carregar cada arquivo CSV em um DataFrame e adicionar à lista
    for arquivo in arquivos_csv:
        caminho_completo = os.path.join(pasta, arquivo)
        df = pd.read_csv(caminho_completo)
        dataframes.append(df)
        print(f'{arquivo} importado com sucesso!')

    # Concatenar todos os DataFrames em um único
    return pd.concat(dataframes, ignore_index=True)

# Caminhos para as duas pastas
pasta1 = 'E:\\Dropbox\\transito\\microdados\\2019'
pasta2 = 'E:\\Dropbox\\transito\\microdados\\2022'

# Importar arquivos de ambas as pastas
df_pasta1 = importar_csv_de_pasta(pasta1)
df_pasta2 = importar_csv_de_pasta(pasta2)

# Concatenar os DataFrames das duas pastas
df = pd.concat([df_pasta1, df_pasta2], ignore_index=True)
print(df.head())

C:\Users\rosal\AppData\Local\Temp/ipykernel_10316/2174447303.py:15: DtypeWarning: Columns (5,8) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(caminho_completo)


2019-01-01_2019-06-30.csv importado com sucesso!
2019-07-01_2019-12-31.csv importado com sucesso!
abr_jun_2022.csv importado com sucesso!
jan_mar_2022.csv importado com sucesso!
jul_set_2022.csv importado com sucesso!
out_dez_2022.csv importado com sucesso!
   Unnamed: 0    datetime_autuacao  \
0         0.0  2019-06-11 11:52:00   
1         1.0  2019-06-11 09:38:00   
2         2.0  2019-04-20 21:36:00   
3         3.0  2019-04-20 06:48:00   
4         4.0  2019-06-26 08:16:00   

                                     status_infracao     pontuacao  \
0  89 - INFRACAO ENCAMINHADA AO DETRAN PARA VALID...       05GRAVE   
1  89 - INFRACAO ENCAMINHADA AO DETRAN PARA VALID...  07GRAVISSIMA   
2  89 - INFRACAO ENCAMINHADA AO DETRAN PARA VALID...       05GRAVE   
3  41 - NOTIFICACAO PG COM REC. EM 2 INSTANCIA IN...       04MEDIA   
4  89 - INFRACAO ENCAMINHADA AO DETRAN PARA VALID...  07GRAVISSIMA   

  especie_veiculo uf_proprietario  valor_infracao  valor_pago data_pagamento  
0      PASSAG

In [6]:
# Converter as colunas de datas para o formato datetime
df['datetime_autuacao'] = pd.to_datetime(df['datetime_autuacao'], errors='coerce')
df['data_pagamento'] = pd.to_datetime(df['data_pagamento'], errors='coerce')

# Criar uma nova coluna 'paid' que indica se a multa foi paga
df['paid'] = df['valor_pago'] > 0

# Calcular o número de dias para pagar, se pago
df['days_to_pay'] = (df['data_pagamento'] - df['datetime_autuacao']).dt.days

# Limpar e selecionar as colunas relevantes
df_cleaned = df[['pontuacao', 'especie_veiculo', 'datetime_autuacao', 'paid', 'days_to_pay']].copy()

# Extrair o mês e ano de autuação
df_cleaned['month_autuacao'] = df_cleaned['datetime_autuacao'].dt.month
df_cleaned['year_autuacao'] = df_cleaned['datetime_autuacao'].dt.year

# Agrupar pelas características e calcular a porcentagem paga, o tempo médio para pagar e a contagem de casos
result_with_year = df_cleaned.groupby(['especie_veiculo', 'pontuacao', 'year_autuacao', 'month_autuacao']).agg(
    count=('pontuacao', 'size'),
    percentage_paid=('paid', 'mean'),
    mean_days_to_pay=('days_to_pay', lambda x: x[x >= 0].mean())  # Considerar apenas dias positivos para o cálculo
).reset_index()

# Converter porcentagem para escala de 0-100 e arredondar valores
result_with_year['percentage_paid'] = (result_with_year['percentage_paid'] * 100).round(2)
result_with_year['mean_days_to_pay'] = result_with_year['mean_days_to_pay'].round(0)

# Exibir o resultado ou salvar em um arquivo CSV
print(result_with_year)

     especie_veiculo     pontuacao  year_autuacao  month_autuacao  count  \
0                  0            00           2019               1      8   
1                  0            00           2019               3      8   
2                  0            00           2019              11      2   
3                  0            00           2019              12      8   
4                  0        03LEVE           2019               7      7   
...              ...           ...            ...             ...    ...   
1323          TRACAO  07GRAVISSIMA           2022               8     99   
1324          TRACAO  07GRAVISSIMA           2022               9    377   
1325          TRACAO  07GRAVISSIMA           2022              10    432   
1326          TRACAO  07GRAVISSIMA           2022              11     52   
1327          TRACAO  07GRAVISSIMA           2022              12     39   

      percentage_paid  mean_days_to_pay  
0              100.00            1216.0  
1  

In [7]:
# Excluir os dataframes que não são mais necessários para liberar memória
del df, df_cleaned

In [8]:
result_with_year.to_csv("resultado_final.csv", index=False)